#  Calibration of two Cameras

Now that the calibration of a camera is already done (calibration.ipynb). It is possible to
calibrate a second camera in the same way. The calibration of each camera is done by
relation to the same object whose acquisition is made simultaneously by the two cameras

In [2]:
import numpy as np
import pandas as pd

In [35]:
def myCalibration(CamG, CamD):
    
    ### This is for left camera 
    np.set_printoptions(suppress=True)
    A = []
    B = []
    for d in CamG:
        A.append([d[0], d[1], d[2], 1, 0, 0, 0, 0, d[3]*d[0]*(-1), d[3]*d[1]*(-1), d[3]*d[2]*(-1)])
        A.append([0, 0, 0, 0, d[0], d[1], d[2], 1,d[4]*d[0]*(-1), d[4]*d[1]*(-1), d[4]*d[2]*(-1)])
        B.append(d[3])
        B.append(d[4])
    B = np.array(B)
    A = np.array(A)
    
    s=np.matmul(np.transpose(A),A)
    s=np.linalg.pinv(s)
    s=np.matmul(s,np.transpose(A))
    S=np.matmul(s,np.transpose(B))
    
    
    M1 = np.array([S[0], S[1], S[2]])
    M2 = np.array([S[4], S[5], S[6]])
    M3 = np.array([S[8], S[9], S[10]])
    M14 = S[3]
    M24 = S[7]
    M34 = 1
    M3t = np.transpose(M3)
    
    u0 = M1@M3t
    v0 = M2@M3t
    
    au = np.linalg.norm(np.cross(M1, M3))
    av = np.linalg.norm(np.cross(M2, M3))
    
    tx = (M14-u0*M34)/au
    ty = (M24-v0*M34)/av
    tz = 1
    r1 = (M1-u0*M3)/au
    r2 = (M2-v0*M3)/av
    r3 = M3
    
    
    MI = np.array([[au, 0, u0, 0],
                   [0, av, v0, 0],
                   [0, 0, 1, 0]])
    ME = np.array([[r1[0], r1[1], r1[2], tx],
                   [r2[0], r2[1], r2[2], ty],
                   [r3[0], r3[1], r3[2], tz]])
    

    
    
    
    
    
    
    
    ### This is for right camera 
    np.set_printoptions(suppress=True)
    AA = []
    BB = []
    for d in CamD:
        AA.append([d[0], d[1], d[2], 1, 0, 0, 0, 0, d[3]*d[0]*(-1), d[3]*d[1]*(-1), d[3]*d[2]*(-1)])
        AA.append([0, 0, 0, 0, d[0], d[1], d[2], 1,d[4]*d[0]*(-1), d[4]*d[1]*(-1), d[4]*d[2]*(-1)])
        BB.append(d[3])
        BB.append(d[4])
    BB = np.array(BB)
    AA = np.array(AA)
    
    ss=np.matmul(np.transpose(AA),AA)
    ss=np.linalg.pinv(ss)
    ss=np.matmul(ss,np.transpose(AA))
    SS=np.matmul(ss,np.transpose(BB))
    
    
    M1t = np.array([SS[0], SS[1], SS[2]])
    M2t = np.array([SS[4], SS[5], SS[6]])
    M3t = np.array([SS[8], SS[9], SS[10]])
    M14t = SS[3]
    M24t = SS[7]
    M34t = 1
    M3tt = np.transpose(M3t)
    
    u0t = M1t@M3tt
    v0t = M2t@M3tt
    
    aut = np.linalg.norm(np.cross(M1t, M3t))
    avt = np.linalg.norm(np.cross(M2t, M3t))
    
    txt = (M14t-u0t*M34t)/aut
    tyt = (M24t-v0t*M34t)/avt
    tzt = 1
    r1t = (M1t-u0t*M3t)/aut
    r2t = (M2t-v0t*M3t)/avt
    r3t = M3t
    
    
    MII = np.array([[aut, 0, u0t, 0],
                   [0, avt, v0t, 0],
                   [0, 0, 1, 0]])
    MEE = np.array([[r1t[0], r1t[1], r1t[2], txt],
                   [r2t[0], r2t[1], r2t[2], tyt],
                   [r3t[0], r3t[1], r3t[2], tzt]])
    
    
    Mat_A=[[M1[0]-np.multiply(M3[0],u0),M1[1]-np.multiply(M3[1],u0),M1[2]-np.multiply(M3[2],u0)],
           [M2[0]-np.multiply(M3[0],v0),M2[1]-np.multiply(M3[1],v0),M2[2]-np.multiply(M3[2],v0)],
           [M1t[0]-np.multiply(M3t[0],u0t),M1t[1]-np.multiply(M3t[1],u0t),M1t[2]-np.multiply(M3t[2],u0t)],
           [M2t[0]-np.multiply(M3t[0],v0t),M2t[1]-np.multiply(M3t[1],v0t),M2t[2]-np.multiply(M3t[2],v0t)]]
    Mat_B=[[M14-u0],[M24-v0],[M14t-u0t],[M24t-v0t]]
    
    
    # We estimate the 3D reconstruction of all the 2D points that were not used for calibration.
    
    
    for i in range(len(Mat_A)):
        X = []
        res = []
        for j in range(len(Mat_A[i])):
            X.append(np.matmul(np.linalg.pinv(Mat_A),np.multiply(-1,Mat_B)))
        res.append(X)
    print(res)
        
    
    
    
    

In [36]:
np.set_printoptions(suppress=True)
mydata = pd.read_csv("mypoints.csv")
dataG=np.array(mydata)
CamG=dataG[0:13]
print(CamG.shape,"\n",CamG)

(13, 7) 
 [[140 20 0 655.0 759.5 391.5 '735.5;;;']
 [120 20 0 839.5 792.5 575.0 '761;;;']
 [140 0 0 639.5 948.0 374.5 '919.5;;;']
 [120 0 0 828.0 972.0 563.0 '938;;;']
 [60 -20 0 1306.5 1207.5 1055.5 '1158.5;;;']
 [40 -20 0 1437.5 1221.5 1195.0 '1170;;;']
 [20 -20 0 1553.9 1234.5 1320.5 '1181.5;;;']
 [60 -40 0 1308.5 1373.5 1057.0 '1324;;;']
 [40 -40 0 1439.5 1378.5 1197.5 '1327.5;;;']
 [20 -40 0 1557.0 1383.0 1324.0 '1331;;;']
 [60 -60 0 1310.5 1541.5 1061.5 '1491.5;;;']
 [40 -60 0 1442.5 1536.5 1201.5 '1486.5;;;']
 [20 -60 0 1559.0 1532.5 1327.5 '1481;;;']]


In [37]:
np.set_printoptions(suppress=True)
mydata = pd.read_csv("mypoints.csv")
dataD=np.array(mydata)
CamD=dataD[13:26]
print(CamG.shape,"\n",CamD)

(13, 7) 
 [[0 -20 20 1766.0 1230.0 1534.5 '1173;;;']
 [0 -20 40 1886.0 1212.5 1645.5 '1152.5;;;']
 [0 -20 60 2019.5 1193.0 1772.0 '1131;;;']
 [0 -40 20 1770.5 1378.5 1538.5 '1323.5;;;']
 [0 -40 40 1890.5 1369.5 1651.0 '1313;;;']
 [0 -40 60 2025.5 1360.5 1779.0 '1301;;;']
 [0 -60 20 1773.0 1529.0 1543.0 '1475.5;;;']
 [0 -60 40 1894.0 1529.0 1655.5 '1475;;;']
 [0 -60 60 2029.0 1530.0 1784.0 '1474.5;;;']
 [0 -120 120 2507.0 2118.0 2263.0 '2083;;;']
 [0 -120 140 2697.5 2150.0 2457.5 '2119.5;;;']
 [0 -140 120 2488.5 2295.5 2248.5 '2267;;;']
 [0 -140 140 2673.0 2336.0 2436.0 '2312.5;;;']]


In [38]:
myCalibration(CamG, CamD)

[[array([[ 178.92718599],
       [ 198.95265091],
       [-275.86504731]]), array([[ 178.92718599],
       [ 198.95265091],
       [-275.86504731]]), array([[ 178.92718599],
       [ 198.95265091],
       [-275.86504731]])]]
